# Set-up


In [129]:
import os
import numpy as np
import pandas as pd

import scipy
import scipy.io
import scipy.stats as stats
import pywt

from PIL import Image
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [130]:
!python --version
print(scipy.version.version)

Python 3.8.5
1.6.0


# Functions

In [131]:
def get_data_from_Drive(path_to_data_folder):
  from google.colab import drive
  drive.mount('/content/drive')
  return get_data_from_folder(os.path.join("/content/drive/MyDrive", path_to_data_folder))  

def get_data_from_folder( path_to_data_folder ):
  #Each class must recide in its own folder.
  #Returns data as (image as nparray, class)
  #gathering classes
  class_titles = os.listdir(path_to_data_folder)
  classes_list = list( ( enumerate( class_titles ) ) )
  ordered_image_names = []
  #placing image, class identifier (from classes_list) pairs in a list named data
  data = []
  number_of_images = 0
  errors = 0 
  for gyro_class in classes_list:
    for image_name in os.listdir( os.path.join( path_to_data_folder, gyro_class[1] ) ):
      number_of_images+=1
      try:
        im = Image.open( os.path.join( path_to_data_folder, gyro_class[1] , image_name)  )
        color_image_nparray = np.asarray( im )
        data.append( [color_image_nparray, gyro_class[0]] )
        ordered_image_names.append(image_name)
      except Exception as error :
        errors+=1
        print( error, image_name)

  return data, class_titles, ordered_image_names, ( number_of_images, errors )

def get_coefficients_flattend( image, wavelet, levels ):
  #image as nparray with color
  #flattens and groups coefficients by orientation and subband
  #returns as  ( cAn, (cHn, cVn, cDn), … (cH1, cV1, cD1)  )
  #                                 . 
  #                                 .     
  #                                 . 
  #for each picture
  #n is the level of decomposition
  
  pil_image = Image.fromarray(image, "RGB")
  pil_gray_image = pil_image.convert("L")
  gray_img = np.asarray( pil_gray_image ) 
  decomposed = pywt.wavedec2( gray_img , wavelet, level=levels)

  coeff=[]
  for subband in range(levels+1):
    if subband==0:
      coeff.append(np.asarray(decomposed[subband]).flatten())
      continue
    orientations=[]
    for orientation in decomposed[subband]:
      orientations.append( np.asarray(orientation).flatten() )
    coeff.append(orientations)    

  return coeff

def qqplots_of_image(image, wavelet, levels, dist):
  
  coeffs=get_coefficients_flattend(image, wavelet, levels)
  alpha=0.2

  fig,axes = plt.subplots(levels+1,3,figsize=(20,20), constrained_layout=True)
  
  #Progress
  print(getattr(dist,"name"), end=": ")
  for m in range(levels+1):
    if m==0:
      sm.qqplot(coeffs[0], dist=dist, fit=True,\
                line="45", linewidth=0.1, alpha=alpha, marker=".", markerfacecolor="black", mew=0, ax=axes[levels-m][0])
      axes[levels-m][0].set_title("Approximation")
      axes[levels-m][0].tick_params(direction="in")
      axes[levels-m][1].remove()
      axes[levels-m][2].remove()
      continue
    for i in range(3):
      if i==0: title="Level "+str(levels-m+1)+": Horizontal"
      if i==1: title="Level "+str(levels-m+1)+": Vertical"
      if i==2: title="Level "+str(levels-m+1)+": Diagonal"

      sm.qqplot(coeffs[m][i], dist=dist, fit=True,\
                line="45", linewidth=0.1, alpha=alpha, marker=".", markerfacecolor="black", mew=0, ax=axes[levels-m][i])
      axes[levels-m][i].set_title(title)
      axes[levels-m][i].tick_params(direction="in")

      #Progress
      print(levels-m+1,i,sep="", end=" ")
  print("")
  
  title="qqplots wavelet={} levels={} dist={}".format(wavelet, levels, getattr(dist,"name"))
  fig.suptitle(title, fontsize=16)
  fig.savefig(title)

  return



# Visualising grouping by distribution parameters

In [143]:
def data_to_coeffs(data, wavelet, levels):
    toReturn=[]
    for image, class_id in data:
        instance = get_coefficients_flattend(image, wavelet, levels), class_id
        toReturn.append(instance)
    return toReturn
#return 2D-array (coeffs, class)

def save_coeffs_for_matlab(data, wavelet, levels, save_folder_path):
    all_coeffs = data_to_coeffs(data, wavelet, levels)
    toSave ={"wavelet":wavelet,\
             "levels":levels,\
             "coeffs":np.array(all_coeffs) }

    scipy.io.savemat(os.path.join(save_folder_path ,"coefficients wt="+wavelet+"levels="+str(levels)+".mat"), toSave)
    return

def load_distribution_parameters_from_matlab(distribution_parameters_path):
    load_dict = scipy.io.loadmat(distribution_parameters_path)

    return load_dict

def prepare_features_for_scikit(dist_params_select_path):\
#path: to dist_paramsSelect    
    load_dict= load_distribution_parameters_from_matlab(dist_params_select_path)
    
    dist_params = load_dict["dist_params"]
    levels      = load_dict["levels"].reshape(-1)[0]
    wavelet     = load_dict["wavelet"]
    selection   = load_dict["selection"]
    print(selection)
    features=[]
    labels=[]
    column_names=[]
    for instance_index in range(dist_params.shape[0]):
        features_from_instance=[]

        for level_index in range(dist_params[instance_index,0].shape[1]):
            a=level_index==0
            b=([0, 0] in selection.tolist())

            if a&b:
                for parameter_index in range(dist_params[instance_index,0][0,0][0,:].shape[0]):
                    features_from_instance.append(dist_params[instance_index,0][0,0][0, parameter_index])
                    
                    if instance_index==0:
                        column_name = "".join(str(a) for a in ["Approximation", " Parameter ", parameter_index+1])
                        column_names.append(column_name)
                continue

            for orientation in range(3):
                if [levels-level_index+1, orientation+1] in selection.tolist():
                    for parameter_index in range(dist_params[instance_index,0][0,level_index][0,:].shape[0]):
                        features_from_instance.append(dist_params[instance_index,0][0,level_index][orientation, parameter_index])
#creating column name
                        if instance_index==0:
                            column_name="".join(str(a) for a in ["Level ",\
                                                                 levels-level_index+1,\
                                                                 " Orientation ",\
                                                                 orientation+1,\
                                                                 " Parameter ",\
                                                                 parameter_index+1])
                            column_names.append(column_name)

        features.append(features_from_instance)
        labels.append(dist_params[instance_index,1].reshape(-1)[0])
    
#dataframe set up
    df = pd.DataFrame(features, columns=column_names)
    df["Labels"] = labels
        
    return df



def class_plot_from_distribution_parameters(distribution_parameters_path,\
                                            save_folder_path,\
                                            data_path="D:\\Work and uni\\University\\ML for food authentication\\Data"):
#dist_params from matlab    
    _ , class_titles, _, _= get_data_from_folder(data_path)
    load_dict = load_distribution_parameters_from_matlab(distribution_parameters_path)
    dist_params = load_dict["dist_params"]
    levels = load_dict["levels"][0][0]
    wavelet = load_dict["wavelet"][0]
    distribution = load_dict["distribution"][0]
    
    fig, ax = plt.subplots(figsize=(10,10))
    title="Plot of distribution parameters for\nWavelet "+wavelet+" | Levels: "+str(levels)+" | Distribution: "+ distribution
    
    N=len(class_titles)
    cmap = plt.cm.jet
    cmaplist = [cmap(i) for i in range(cmap.N)]
    cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)
    bounds = np.linspace(0,N,N+1)
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    
    for instance, label in dist_params:
        for level in np.reshape(instance, -1):
            for orientation in level:
                scat = ax.scatter(orientation[0], orientation[1], c=np.reshape(label,-1)[0], s=50, alpha=1, cmap=cmap, norm=norm)
    ax.set_title(title)
    
    ax.set_xlabel("Ditribution Parameter 1")
    ax.set_ylabel("Ditribution Parameter 2")
    ax.tick_params(direction="in")
    cb = plt.colorbar(scat, spacing='proportional',ticks=bounds)
    cb.set_label("Classes")
    
    fig.savefig(os.path.join(save_folder_path, "wt= "+wavelet+" levels= "+str(levels)+" dist="+load_dict["distribution"][0]+".jpeg"), dpi=200)
    
    return

# Testing - Using

In [140]:
data , class_titles, ordered_image_names, results= get_data_from_folder("D:\\Work and uni\\University\\ML for food authentication\\Data")
print("RESULTS\n",results,"\n==========================")
print("CLASS,titles\n",class_titles,"\n===========================")


cannot identify image file 'D:\\Work and uni\\University\\ML for food authentication\\Data\\cichorium intubus pollen x 400\\Cichorium Intybus (Asteraceae)_1_bak.tif' Cichorium Intybus (Asteraceae)_1_bak.tif
cannot identify image file 'D:\\Work and uni\\University\\ML for food authentication\\Data\\cichorium intubus pollen x 400\\Cichorium Intybus (Asteraceae)_3_bak.tif' Cichorium Intybus (Asteraceae)_3_bak.tif
cannot identify image file 'D:\\Work and uni\\University\\ML for food authentication\\Data\\cichorium intubus pollen x 400\\Cichorium Intybus (Asteraceae)_bak.tif' Cichorium Intybus (Asteraceae)_bak.tif
RESULTS
 (56, 3) 
CLASS,titles
 ['castanea 2_pio synithismeni morfi', 'castanea sp', 'cichorium intubus pollen x 400', 'citrus sp', 'eukalyptus sp', 'hedera helix', 'helianthus', 'heliotropium', 'hibiscous', 'inula spinoza', 'origanum', 'pinus', 'sonchus', 'thymus sp pollen 20-01-21', 'tribulus sp'] 


In [141]:
#wavevelet_decomposition, producing coefficients
wavelets = ["db4","db8","db12"]
levels_list = [3]
save_folder_path="D:\\Work and uni\\University\\ML for food authentication\\Results_fitting\\coefficients\\"
for wavelet in wavelets:
    for levels in levels_list:
        save_coeffs_for_matlab(data, wavelet, levels, save_folder_path)
#now run matlab code

<ipython-input-139-de7e1e1f4911>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  "coeffs":np.array(all_coeffs) }


In [6]:
dist_paramsSelect_folder_path = "D:\\Work and uni\\University\\ML for food authentication\\Results_fitting\\dist_paramsSelect\\"
dist_paramsSelect_name="dist_paramsSelect wt=db4 levels=5 dist=Stable.mat"
dist_paramsSelect_path=os.path.join(dist_paramsSelect_folder_path, dist_paramsSelect_name)

df = prepare_features_for_scikit(dist_paramsSelect_path)
df

[[5 1]
 [5 2]
 [5 3]
 [0 0]]


,Approximation Parameter 1,Approximation Parameter 2,Approximation Parameter 3,Approximation Parameter 4,Level 5 Orientation 1 Parameter 1,Level 5 Orientation 1 Parameter 2,Level 5 Orientation 1 Parameter 3,Level 5 Orientation 1 Parameter 4,Level 5 Orientation 2 Parameter 1,Level 5 Orientation 2 Parameter 2,Level 5 Orientation 2 Parameter 3,Level 5 Orientation 2 Parameter 4,Level 5 Orientation 3 Parameter 1,Level 5 Orientation 3 Parameter 2,Level 5 Orientation 3 Parameter 3,Level 5 Orientation 3 Parameter 4,Labels
0,1.606220,-1.000000,224.672103,5162.071497,1.407513,0.034562,79.588095,-2.831343,1.473640,-0.050537,82.272178,0.243778,1.592530,-0.021161,66.084814,-0.155974,0
1,1.501644,-0.979266,231.070882,5184.793593,1.464732,-0.025149,76.083451,2.437430,1.495908,-0.055204,84.404506,5.989328,1.606427,0.023567,64.229407,1.140553,0
2,1.715838,-0.956886,535.128610,5070.437766,1.777361,-0.032170,313.917257,-0.085759,1.730021,-0.068143,307.708435,5.499434,1.724315,0.017293,161.373020,-2.836815,1
3,1.377655,-0.949340,290.730165,5523.521294,0.944074,0.000047,81.939012,1.216845,0.987393,-0.053258,94.125266,0.567655,1.031157,-0.000042,64.779516,-0.430016,1
4,1.467276,-0.942497,508.340338,5231.449779,1.140245,0.006303,171.591499,2.306258,1.239728,-0.003234,202.919675,1.880082,1.165598,0.006066,97.549836,-0.415232,1
5,1.671833,-0.976683,465.162671,5105.482962,1.664928,-0.033451,283.921496,-4.693159,1.612296,-0.023553,273.392018,11.178851,1.689693,0.001665,173.869855,-1.234692,1
6,1.595454,-0.703600,405.679797,5282.204367,1.252904,-0.002295,186.989563,3.984176,1.399818,-0.036854,231.646690,13.247066,1.414795,-0.025357,141.560344,-0.683747,1
7,1.747572,-0.955325,160.994667,6411.866663,1.323127,0.026252,19.974934,-0.571790,1.324452,0.010018,20.579295,0.015237,1.243427,0.001383,11.436436,0.083448,3
8,1.596240,-0.994006,152.219745,6154.073050,1.417666,0.047941,15.940619,-1.014714,1.383955,0.032446,16.448153,-0.235306,1.299488,-0.043208,10.890075,0.031593,4
9,1.091410,-0.962158,235.176572,5284.068854,0.728161,0.001509,18.540153,0.087277,0.773428,-0.054029,19.723246,0.954704,0.696099,-0.008735,9.772423,0.009971,5


In [ ]:
#now load matlab results
load_dict = scipy.io.loadmat("dist_params wt=db4 levels=5 dist=Normal.mat")
dist_params = load_dict["dist_params"]

In [ ]:
usable = dist_params
print(usable.shape)
toHist = []
def plot_fitted_params(parameters):
        continue

def hist_of_parameters(parameters):
  alpha=0.2

  fig,axes = plt.subplots(levels+1,3,figsize=(20,20), constrained_layout=True)
  
  #Progress
  print(getattr(dist,"name"), end=": ")
  for m in range(levels+1):
    if m==0:
      sm.qqplot(coeffs[0], dist=dist, fit=True,\
                line="45", linewidth=0.1, alpha=alpha, marker=".", mfc="black", mew=0, ax=axes[levels-m][0])
      axes[levels-m][0].set_title("Approximation")
      axes[levels-m][0].tick_params(direction="in")
      axes[levels-m][1].remove()
      axes[levels-m][2].remove()
      continue
    for i in range(3):
      if i==0: title="Level "+str(levels-m+1)+": Horizontal"
      if i==1: title="Level "+str(levels-m+1)+": Vertical"
      if i==2: title="Level "+str(levels-m+1)+": Diagonal"

      sm.qqplot(coeffs[m][i], dist=dist, fit=True,\
                line="45", linewidth=0.1, alpha=alpha, marker=".", mfc="black", mew=0, ax=axes[levels-m][i])
      axes[levels-m][i].set_title(title)
      axes[levels-m][i].tick_params(direction="in")

      #Progress
      print(levels-m+1,i,sep="", end=" ")
  print("")
  
  title="qqplots wavelet={} levels={} dist={}".format(wavelet, levels, getattr(dist,"name"))
  fig.suptitle(title, fontsize=16)
  fig.savefig(title)

  return

In [ ]:
#single Class Plot
dist_params_folder_path = "D:\\Work and uni\\University\\ML for food authentication\\Results_fitting\\dist_params\\"
save_folder_path = "D:\\Work and uni\\University\\ML for food authentication\\Results_fitting\\Class Plots\\"
dist_params_name="dist_params wt=db4 levels=6 dist=Normal.mat"
dist_params_path=os.path.join(dist_params_folder_path, dist_params_name)

class_plot_from_distribution_parameters(dist_params_path, save_folder_path)

#all available dist_params Plots
"""for dist_params_name in  os.listdir( dist_params_folder_path ):
    print(dist_params_name)
    dist_params_path=os.path.join(dist_params_folder_path, dist_params_name)
    class_plot_from_distribution_parameters(dist_params_path, save_folder_path)

"""


In [ ]:
#qqplots of image 2
print(ordered_image_names[7])
plt.imshow(data[7][0])
qqplots_of_image(data[2][0], "db12", 5, stats.norm)

# Visualisation of wavelet decomposition

In [ ]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import statsmodels.api as sm
import scipy.stats as stats


n=3
im = Image.open("/content/drive/MyDrive/Copy of origanum_bak.tif")
plt.imshow(np.asarray(im))
plt

gray_img = im.convert("L")
plt.figure()
plt.imshow(np.asarray(gray_img), cmap=plt.get_cmap('gray'))
gray_array = np.asarray(gray_img)

C = pywt.wavedec2( gray_array , 'db1', level=n)

C[0] /= np.abs(C[0]).max()
for m in range(n):
  if m==0:
    continue
  for mat in C[m]:
    mat /= np.abs(mat).max()

arr , slices = pywt.coeffs_to_array(C)

plt.figure(figsize=[12,12])
plt.imshow(arr, cmap=plt.get_cmap('gray'), vmin=-0.25, vmax=0.75 )
plt.savefig("hooray.jpeg")

a=[1,2,['name', im]]
plt.figure()
plt.imshow(np.asarray(a[2][1]))

In [16]:
two_classes= df[df["Labels"]==13]
two_classes = two_classes.append(df[df["Labels"]==9])
two_classes

,Approximation Parameter 1,Approximation Parameter 2,Approximation Parameter 3,Approximation Parameter 4,Level 5 Orientation 1 Parameter 1,Level 5 Orientation 1 Parameter 2,Level 5 Orientation 1 Parameter 3,Level 5 Orientation 1 Parameter 4,Level 5 Orientation 2 Parameter 1,Level 5 Orientation 2 Parameter 2,Level 5 Orientation 2 Parameter 3,Level 5 Orientation 2 Parameter 4,Level 5 Orientation 3 Parameter 1,Level 5 Orientation 3 Parameter 2,Level 5 Orientation 3 Parameter 3,Level 5 Orientation 3 Parameter 4,Labels
26,1.871821,-1.0,116.281749,3664.931219,1.031322,0.000021,5.299083,-0.091975,1.131425,-0.011476,5.985789,-0.038894,1.031282,-0.000017,2.493990,0.061276,13
27,1.765869,-1.0,125.365605,3634.243796,1.091893,-0.000182,7.280078,-0.166511,1.079629,0.012280,8.585587,-0.425439,1.031256,-0.000024,3.717049,0.031853,13
28,1.832649,-1.0,123.470711,3621.328428,1.031344,0.000041,5.799413,-0.087176,1.031326,0.000009,6.468863,-0.048073,1.031011,-0.079527,2.788087,0.014565,13
29,1.711732,-1.0,128.234151,3636.454800,1.031329,-0.000021,5.796844,-0.120605,1.125662,0.029928,6.293528,0.084206,1.031311,0.000009,2.648860,0.100463,13
30,1.877317,-1.0,121.094253,3644.063387,1.289277,-0.010398,4.930479,-0.211475,1.309328,0.039284,5.498458,-0.242713,1.298018,0.007155,2.283819,0.068374,13
31,1.988187,-1.0,111.049454,3644.422077,1.335789,-0.006893,5.055829,-0.149746,1.320999,0.036421,5.395905,-0.288540,1.360605,0.052958,2.363227,-0.036874,13
32,1.959750,-1.0,108.451010,3658.306847,1.291909,0.011005,5.070198,-0.178046,1.349376,0.022621,5.721281,-0.204170,1.330813,0.001568,2.472140,0.034798,13
33,1.870520,-1.0,115.637768,3637.255042,1.191047,-0.016202,5.331066,-0.142583,1.270434,0.004860,5.775128,-0.002070,1.241529,-0.023287,2.605336,0.055779,13
34,1.985982,-1.0,108.658347,3640.558704,1.276379,-0.033152,4.782457,-0.102666,1.323861,0.033813,5.472459,-0.114188,1.288333,0.063211,2.161416,-0.057051,13
35,1.984507,-1.0,110.324010,3631.427863,1.302968,-0.013100,4.464636,-0.096822,1.394967,0.018527,5.266321,-0.165091,1.441820,0.031701,2.121905,0.001944,13


In [18]:
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Classes 9 and 13
two_classes= df[df["Labels"]==13]
two_classes = two_classes.append(df[df["Labels"]==9])
two_classes

A=two_classes.to_numpy()
x=A[:,:-1]
y=A[:,-1]
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.5, random_state = 30)
model=pipeline.Pipeline((\
                         ("scaler",StandardScaler() ),\
                         ("svm_clf", svm.SVC(kernel='rbf', gamma=0.009, C=1, decision_function_shape='ovo'))
                        
                        ))

model.fit(X_train, y_train)
print("Validation Accuracy" ,model.score(X_test, y_test))
print("Train Accuracy" ,model.score(X_train, y_train))


Validation Accuracy 0.7647058823529411
Train Accuracy 0.7647058823529411


In [113]:
#randomly chooses images of the larger class to balance out dataset
def balance_classes2(two_classes):
    number_of_instances_of_small_class = two_classes['Labels'].value_counts(sort=True).iloc[1]
    label_of_large_class = two_classes['Labels'].value_counts(sort=True).index[0]
    label_of_small_class = two_classes['Labels'].value_counts(sort=True).index[1]

    to_append_small = two_classes[two_classes["Labels"]==label_of_small_class]
    to_append_large = two_classes[two_classes["Labels"]==label_of_large_class].sample(number_of_instances_of_small_class)

    two_classes_balanced = pd.DataFrame(to_append_small)
    two_classes_balanced = two_classes_balanced.append(to_append_large)
    return two_classes_balanced


In [146]:
#gets 2 larger classes
def get_larger_classes(dist_paramsSelect_name):
    #ready dataframe
    dist_paramsSelect_folder_path = "D:\\Work and uni\\University\\ML for food authentication\\Results_fitting\\dist_paramsSelect\\"    
    dist_paramsSelect_path=os.path.join(dist_paramsSelect_folder_path, dist_paramsSelect_name)

    df = prepare_features_for_scikit(dist_paramsSelect_path)
    

    #keeping 2 top population classes
    class_population=df['Labels'].value_counts(sort=True)
    two_classes= df[df["Labels"]==class_population.index[0]]
    two_classes = two_classes.append(df[df["Labels"]==class_population.index[1]])
    return two_classes,df

In [153]:
#balance and cross-validate
def balance_and_cv(two_classes):
    A=balance_classes2(two_classes).to_numpy()
    x=A[:,:-1]
    y=A[:,-1]

    model=pipeline.Pipeline((\
                             ("scaler",StandardScaler() ),\
                             ("svm_clf", svm.SVC(kernel='rbf', gamma=0.01, C=1, decision_function_shape='ovo'))\
                            ))

    scores = cross_val_score(model, x, y, cv=4)
    return scores

In [154]:
#two balanced classes
results=pd.DataFrame(columns=["Mean Accuracy", "Standard Deviation"])
larger_classes, df = get_larger_classes("dist_paramsSelect wt=db4 levels=5 dist=Stable.mat")
for i in range(20):
    scores=balance_and_cv(larger_classes)
    results = results.append(pd.DataFrame({"Mean Accuracy":[scores.mean()], "Standard Deviation":[scores.std()]}))
    
results

[[5 1]
 [5 2]
 [5 3]
 [0 0]]


,Mean Accuracy,Standard Deviation
0,1.0000,0.000000
0,1.0000,0.000000
0,0.8125,0.207289
0,1.0000,0.000000
0,0.8750,0.216506
0,1.0000,0.000000
0,1.0000,0.000000
0,1.0000,0.000000
0,1.0000,0.000000
0,0.9375,0.108253


In [155]:
#two balanced classes
results=pd.DataFrame(columns=["Mean Accuracy", "Standard Deviation"])
larger_classes, df = get_larger_classes("dist_paramsSelect wt=db4 levels=3 dist=Stable.mat")
for i in range(20):
    scores=balance_and_cv(larger_classes)
    results = results.append(pd.DataFrame({"Mean Accuracy":[scores.mean()], "Standard Deviation":[scores.std()]}))
    
results

[[1 1]
 [1 2]
 [1 3]
 [2 1]
 [2 2]
 [2 3]
 [3 1]
 [3 2]
 [3 3]]


,Mean Accuracy,Standard Deviation
0,1.0,0.0
0,1.0,0.0
0,1.0,0.0
0,1.0,0.0
0,1.0,0.0
0,1.0,0.0
0,1.0,0.0
0,1.0,0.0
0,1.0,0.0
0,1.0,0.0
